In [ ]:
!pip install -q opencv-python
!pip install flask-ngrok

In [ ]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm
from tqdm.autonotebook import tqdm, trange

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Download model from TF Hub and check out inference code from GitHub
!wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
!git clone https://github.com/tensorflow/examples.git
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=10):
  """Runs detection on an input image.
 
  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.
 
  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape
 
  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)
 
  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person

In [ ]:

def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [ ]:

class MoveNetPreprocessor(object):
  """Helper class to preprocess pose sample images for classification."""
 
  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    #Creates a preprocessor to detection pose from images and save as CSV.
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()
 
    # Get list of pose classes and print image statistics
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder) if not n.startswith('.')]
        )
    
  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    #Preprocesses images in the given folder.

    # Loop through the classes and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)
 
      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, 
                                    delimiter=',', 
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0
 
        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape
          
          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)
          
          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:
            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1

          # Draw the prediction result on top of the image for debugging later
          output_overlay = draw_prediction_on_image(
              image.numpy().astype(np.uint8), person, 
              close_figure=True, keep_input_size=True)
        
          # Write detection result into an image file
          output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
        
          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(np.str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))
      
    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    #List of classes found in the training dataset."""
    return self._pose_class_names
  
  def _all_landmarks_as_dataframe(self):
    #"""Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)
      
      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '') 
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)
 
    list_name = [[bodypart.name + '_x', bodypart.name + '_y', 
                  bodypart.name + '_score'] for bodypart in BodyPart] 
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i] 
                  for i in range(len(header_name))}
 
    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

In [ ]:
import os
import difflib
import pandas as pd
df = pd.DataFrame()
df3 = pd.DataFrame()

def string_similarity(str1, str2):
    result =  difflib.SequenceMatcher(a=str1.lower(), b=str2.lower())
    return result.ratio()
def calculation(heights,imgroute):
  z = imgroute
  image = tf.io.read_file(z)
  height = heights
  image = tf.io.decode_jpeg(image)
  pheight = image.get_shape()[0]
  person = detect(image)
  keys = []
  #Y-axis Distortion Correction
  def ydistortionWrapper(cheight, cdistance, sheight, numpixels):
    def ydistortion(pycoord):
      a1 =np.arctan2(cheight,cdistance)
      a2= np.arctan2((sheight-cheight),cdistance)
      p2a = numpixels/(a1+a2)
      a3 = (pheight-pycoord)/p2a
      
      return cdistance*(np.tan(a1)-np.tan(a1-a3))
    return ydistortion
  #X-axis Distortion Correction
  def xdistortionWrapper(cheight, cdistance, sheight, numpixels):
    def xdistortion(pxcoord):
      a1 =np.arctan2(cheight,cdistance)
      a2= np.arctan2((sheight-cheight),cdistance)
      p2a = numpixels/(a1+a2)
      a3 = pxcoord/p2a
      return pxcoord*cdistance/p2a
    return xdistortion
  #Distance Calculation
  def distance1(x1, y1, x2, y2):
    ydis = ydistortionWrapper(62,111,heights,pheight)
    xdis = xdistortionWrapper(62,111,heights,pheight)
    v = np.sqrt(pow(xdis(x1)-xdis(x2),2)+pow(ydis(y1)-ydis(y2),2))
    return v
  #Index the points to an array 
  while(len(keys)==0):
    for z1 in range(len(person.keypoints)):
      if(z1 in [5,6,7,8,9,10,11,12,13,14,15,16]):
        keys.append(person.keypoints[z1])

  d1 = distance1(keys[6].coordinate.x,keys[6].coordinate.y,keys[8].coordinate.x,keys[8].coordinate.y)

  d2 = distance1(keys[7].coordinate.x,keys[7].coordinate.y,keys[9].coordinate.x,keys[9].coordinate.y)

  e1 = distance1(keys[1].coordinate.x,keys[1].coordinate.y,keys[3].coordinate.x,keys[3].coordinate.y)+distance1(keys[3].coordinate.x,keys[3].coordinate.y,keys[5].coordinate.x,keys[5].coordinate.y)
  e2 = distance1(keys[0].coordinate.x,keys[0].coordinate.y,keys[2].coordinate.x,keys[2].coordinate.y)+distance1(keys[2].coordinate.x,keys[2].coordinate.y,keys[4].coordinate.x,keys[4].coordinate.y)

  f1 = distance1(keys[0].coordinate.x,keys[0].coordinate.y,keys[1].coordinate.x,keys[1].coordinate.y)
  b1 = distance1(0,pheight,0,keys[0].coordinate.y)
  b2 = distance1(0,pheight,0,keys[1].coordinate.y)
  c2 = distance1(keys[9].coordinate.x,keys[9].coordinate.y,keys[11].coordinate.x,keys[11].coordinate.y)+d2
  c1 =distance1(keys[8].coordinate.x,keys[8].coordinate.y,keys[10].coordinate.x,keys[10].coordinate.y)+d1

  pred = [(((b1+b2)/2)),(((c1+c2)/2)),(((d1+d2)/2)),(((e1+e2)/2)),(f1)]
  #Return prediction
  return pred
      

In [ ]:
from __future__ import print_function
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import sqlite3 as lite

import matplotlib.pyplot as plt
def analyze(height, imgroute):
  
  with open('/content/file.pkl', 'rb') as fid:
     reg = pickle.load(fid)
     ogpredict = [calculation(height,imgroute)]
     prediction=reg.predict(ogpredict)
     final_prediction=[height,prediction[0][0],prediction[0][1],prediction[0][2],prediction[0][3]]
     return final_prediction

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2D66J6v7V2IxY0Uiz37VSX8XoWg_381XwdRPEnUY2kUwq52fw

In [ ]:
import cv2
import os
import shutil
import csv
from flask_ngrok import run_with_ngrok
import csv

import werkzeug
from flask import Flask, flash, jsonify, redirect, render_template, request, session
import secrets
f = open("temp.csv", "w")
# Configure application
app = Flask(__name__)
run_with_ngrok(app)   
# Ensure templates are auto-reloaded
UPLOAD_FOLDER  = '/content'
ALLOWED_EXTENSIONS = {'png','jpg','jpeg','gif','mp4'}
# Ensure templates are auto-reloaded
app.config["TEMPLATES_AUTO_RELOAD"] = True
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER
app.config['SECRET_KEY'] = secrets.token_urlsafe(16)


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1]
@app.route("/", methods=["GET", "POST"])
def index(hf =[]):
    

    if request.method == "POST":
        input = request.form.get("height")
        input = int(input)
        if 'file' not in request.files:
          print('No File Part')
          return redirect(request.url)
        file = request.files['file']
        if file.filename =='':
          print('No Selected File')
          return redirect(request.url)
        if file and allowed_file(file.filename):
          file.save(os.path.join(app.config['UPLOAD_FOLDER'],file.filename))
          print('success')
          final_result = analyze(input,str(UPLOAD_FOLDER+"/"+file.filename))
      
          f = open("temp.csv", "w")
          f.write(str(final_result[0])+","+str(final_result[1])+","+str(final_result[2])+","+str(final_result[3])+","+str(final_result[4]))

          f.close()

          return redirect("/")
    
    else:
        f = open("temp.csv","r")
        final_result = f.read().split(",")
        print(hf)
        return render_template("index.html",final_result = final_result)
        f.close()
if __name__=='__main__':
  app.run()